In [1]:
import numpy as np
import tensorflow as tf
#import maxout
import highway_maxout as hmn
import utils
import dataset as ds

In [2]:
#======= FLAGS ==========
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('maxout_layer_size', 8, 'Maxout layer size')
tf.app.flags.DEFINE_integer('maxout_pooling_size', 16, 'Maxout pooling size')
tf.app.flags.DEFINE_integer('lstm_size', 200, 'LSTM cell internal size')
tf.app.flags.DEFINE_string('log_path', '/tmp/dcn', 'logs location')


In [3]:
# remove all variables
tf.reset_default_graph();

lstm_size = 200
word_vector_size = 300
maxout_pooling_size = 16
max_decoder_iterations = 4
maxout_layer_size = 8;
max_epoch = 1;
max_sequence_length = 600
#training_set_size = 100;

# 
question_ph = tf.placeholder(tf.float32, [1, max_sequence_length, word_vector_size], name="q_input")
document_ph = tf.placeholder(tf.float32, [1, max_sequence_length, word_vector_size], name="d_input")


with tf.name_scope('ENCODER'):
    # LSTM cell initialization
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)


# LSTM cells for Bi-LSTM for COATINATION ENCODER
with tf.name_scope('COATTENTION_ENCODER'):
    lstm_cenc_fw = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    lstm_cenc_bw = tf.contrib.rnn.BasicLSTMCell(lstm_size)

# create lstm cell for DYNAMIC POINTING DECODER
lstm_dec = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# get lstm initial state of zeroes
#lstm_dec_state = lstm_dec.zero_state(1, tf.float32)
start_pos = 0; # generate random between (0, document_size-1)
end_pos = 0;   # generate random between (0, document_size-1)

# create sentinel vector variable for both encodings 
#with tf.variable_scope("scope1") as scope:
sentinel_q = tf.get_variable("sentinel_q", [ lstm_size , 1], initializer = tf.random_normal_initializer())
sentinel_d = tf.get_variable("sentinel_d", [ lstm_size , 1], initializer = tf.random_normal_initializer()) 

# optimizer

optimizer = tf.train.AdamOptimizer()
    

In [4]:

# r = lstm(inputs = tf.convert_to_tensor([[1,2], [2,3]], dtype=tf.float32), state = zero_state_q)

def length(sequence):
  used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

'''
transform tensor of shape [1, question_size, word_vector_size] to list of tensors of shape [1, word_vector_size]
of length question_size. first dimenstion is batch size = 1
'''

#print(tf.shape(question_ph)[1])
#question_input = tf.unstack(question_ph, max_sequence_length, 1)
#document_input = tf.unstack(document_ph, max_sequence_length, 1)
#print(x)

# we use the same LSTM for both encodings
#outputs_q, state_q = tf.contrib.rnn.static_rnn(lstm, inputs = question_input, sequence_length = length(question_ph), dtype=tf.float32)
#outputs_d, state_d = tf.contrib.rnn.static_rnn(lstm, inputs = document_input, sequence_length = length(document_ph), dtype=tf.float32)

with tf.name_scope('ENCODER'):
    outputs_q, state_q = tf.nn.dynamic_rnn(lstm, inputs = question_ph, sequence_length = length(question_ph), dtype=tf.float32)
    outputs_d, state_d = tf.nn.dynamic_rnn(lstm, inputs = document_ph, sequence_length = length(document_ph), dtype=tf.float32)


# "squeeze" transforms list of tensors of shape [1, lstm_size] of length L to tensor of shape [L, lstm_size]
que_enc = tf.transpose(tf.squeeze(outputs_q))
doc_enc = tf.transpose(tf.squeeze(outputs_d))
document_size =tf.shape(doc_enc)[1]


# append sentinel vector for both encodings 
doc_enc_sentinel = tf.concat([doc_enc, sentinel_d], axis = 1)
que_enc_sentinel = utils.non_linear_projection(tf.concat([que_enc, sentinel_q], axis = 1))


# ===================  COATTENTION ENCODER ===================
with tf.name_scope('COATTENTION_ENCODER'):
    # L \in R(doc_size + 1) x (que_size + 1)
    L = tf.matmul(doc_enc_sentinel, que_enc_sentinel, transpose_a = True)
    A_Q = tf.nn.softmax(L, 0)
    A_D = tf.nn.softmax(tf.transpose(L), 1)
    C_Q = tf.matmul(doc_enc_sentinel, A_Q)
    # C_D \in R_2*lstm_size x (doc_size + 1)
    C_D = tf.matmul(tf.concat([que_enc_sentinel, C_Q], axis = 0), A_D)

    # bi_lstm_input = tf.unstack(tf.reshape(tf.transpose(tf.concat([doc_enc_sentinel, C_D], axis = 0)), [max_sequence_length + 1, 1, 3*lstm_size]))
    # TODO Q: would we use single cell of two different
    bi_lstm_input = tf.concat([doc_enc_sentinel, C_D], axis = 0)
    bi_lstm_input = tf.transpose(bi_lstm_input)
    bi_lstm_input = tf.reshape(bi_lstm_input, [1, max_sequence_length + 1, 3*lstm_size])
    test = tf.placeholder(tf.float32, [1,12,30]);
    outputs_bi, output_state = tf.nn.bidirectional_dynamic_rnn(
        cell_fw = lstm_cenc_fw, 
        cell_bw = lstm_cenc_fw,
      #  cell_bw = lstm_cenc_bw,
        inputs = bi_lstm_input,
      #  sequence_length = [12],
        dtype=tf.float32
    )

    # we take first because of we feed to bi-RNN only one sentence
    outputs_bi = tf.concat(outputs_bi, axis=2)[0]
    print(outputs_bi)
    U = tf.slice(outputs_bi, [0,0], [document_size, 2*lstm_size])
    U = tf.transpose(U)
#print(U)


Tensor("COATTENTION_ENCODER_1/strided_slice:0", shape=(601, 400), dtype=float32)


In [5]:
# ===================== DYNAMIC POINTING DECODER =============


#scope = tf.get_variable_scope()
#u_t = get_scope_variable(scope, 'hmn_u_t', [2*lstm_size, 1]) 
#h_i = get_scope_variable(scope, 'hmn_h_i', [lstm_size, 1 ]) 
#u_s_i = get_scope_variable(scope, 'hmn_u_s_i', [2*lstm_size, 1])
#u_e_i = get_scope_variable(scope, 'hmn_u_e_i', [2*lstm_size, 1])


#m_3 = HMN(U, h_i, u_s_i, u_e_i)
#print(m_3)

# returns tuple (scores_start, scores_end, strart_pos, start_end, new_lstm_state)
def decoderIteration(U, lstm_state, start_pos, end_pos):
    with tf.name_scope('Decoder_Iteration'):
        with tf.name_scope('Next_Start'):
            scores_start = hmn.HMN(U, 
                               tf.transpose(lstm_state.h), 
                               tf.slice(U, [0, start_pos], [lstm_size*2, 1]) ,
                               tf.slice(U, [0, end_pos], [lstm_size*2, 1],) , 
                               'start',
                                FLAGS)

            new_start_pos = tf.to_int32(tf.argmax(scores_start, 0))

        #print(lstm_state)
        with tf.name_scope('Next_End'):
            scores_end = hmn.HMN(U, 
                             tf.transpose(lstm_state.h), 
                             tf.slice(U, [0, new_start_pos], [lstm_size*2, 1],) ,
                             tf.slice(U, [0, end_pos], [lstm_size*2, 1],), 
                            'end',
                            FLAGS)
            new_end_pos = tf.to_int32(tf.argmax(scores_end, 0))
        
        with tf.name_scope('LSTM_State_Update'):
            lstm_input = tf.concat(
                [tf.slice(U, [0, new_start_pos], [lstm_size*2, 1], name='slice-5'), tf.slice(U, [0, new_end_pos], [lstm_size*2, 1])],
                axis = 0
            )
            output, new_lstm_state = lstm_dec(tf.reshape(lstm_input, [1, lstm_size*4]), lstm_state)
        
        #print(new_lstm_state)
        return scores_start, scores_end, new_start_pos , new_end_pos, new_lstm_state



#print(lstm_dec_state)

with tf.name_scope('DYNAMIC_POINTING_DECODER'):
    
    start_pos = 0;
    end_pos = 0;
    sum_start_scores = tf.zeros([1, document_size])
    sum_end_scores = tf.zeros([1, document_size])
    lstm_dec_state = lstm_dec.zero_state(1, tf.float32)
    
    for step in range(max_decoder_iterations):
        scores_start, scores_end, new_start_pos, new_end_pos, lstm_dec_state = decoderIteration(U, lstm_dec_state, start_pos, end_pos)
        sum_start_scores = tf.add(sum_start_scores, scores_start)
        sum_end_scores   = tf.add(sum_end_scores, scores_end)
        if new_start_pos == start_pos and end_pos == new_end_pos : break
        start_pos = new_start_pos
        end_pos = new_end_pos

    
# loss and train step
start_end_true = tf.placeholder(tf.int32, [2]);
#end_true = tf.placeholder(tf.int32, ());
onehot_labels = tf.one_hot(start_end_true, document_size)
with tf.name_scope('Loss'):
    sum_loss = tf.losses.softmax_cross_entropy(
        onehot_labels,
        tf.concat([sum_start_scores, sum_end_scores], axis=0))


tf.summary.scalar('loss', sum_loss)
with tf.name_scope('Train'):
    train_step = optimizer.minimize(sum_loss)


In [ ]:
#=========== Training ==================

dataset = ds.getDataset(["./train_train_task_b.csv"], max_sequence_length)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
summary_op = tf.summary.merge_all()

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(FLAGS.log_path, sess.graph)
    for epoch_ in range(max_epoch):
        for step_ in range(7000):
            start_true, end_true, doc, que, doc_v, que_v = sess.run(next_element)
            
            if step_ % 100 == 99:
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                _,loss,stat = sess.run(
                    (train_step, sum_loss, summary_op), 
                    feed_dict={question_ph: [que_v], document_ph: [doc_v], start_end_true: [start_true, end_true]},
                    options=run_options,
                    run_metadata=run_metadata
                )
                print(step_, loss)
                writer.add_run_metadata(run_metadata, 'step%d' % step_)
                writer.add_summary(stat,  step_)
            else: 
                _,loss, stat = sess.run(
                    (train_step, sum_loss, summary_op), 
                    feed_dict={question_ph: [que_v], document_ph: [doc_v], start_end_true: [start_true, end_true]}
                )
                print(step_, loss)
                writer.add_summary(stat,  step_)
                
    print('End')


0 6.45938
1 6.38718
2 6.59994
3 7.34657
4 7.27166
5 6.96637
6 6.52183
7 6.85479
8 6.62029
9 5.29043
10 7.0155
11 5.7589
12 7.79244
13 6.26254
14 7.3376
15 7.66651
16 6.3598
17 5.33968
18 6.86679
19 6.44198
20 6.55311
21 5.34536
22 6.48824
23 5.26756
24 5.40209
25 7.12319
26 6.89291
27 6.10444
28 5.41035
29 5.40294
30 5.23661
31 6.36622
32 8.71191
33 4.17728
34 5.47509
35 8.99654
36 6.79967
37 4.50886
38 7.04263
39 6.21332
40 4.39496
41 8.79247
42 8.62935
43 5.60814
44 5.71591
45 6.6762
46 7.29791
47 5.28108
48 5.7097
49 6.21832
50 8.39337
51 8.1439
52 4.81459
53 6.68883
54 5.69541
55 3.8126
56 4.53181
57 6.72162
58 4.8155
59 5.68192
60 5.88921
61 2.67836
62 6.56872
63 11.6632
64 9.56131
65 6.63882
66 7.23728
67 6.21503
68 8.0196
69 6.62954
70 7.9546
71 6.45439
72 6.14967
73 6.3173
74 6.22205
75 5.71898
76 5.17362
77 6.1592
78 5.02904
79 7.06811
80 6.73253
81 4.13729
82 5.46089
83 5.39036
84 4.20929
85 4.64532
86 6.67319
87 3.63488
88 4.74851
89 5.80312
90 4.01924
91 6.74559
92 5.82098


In [42]:
print(tf.nn.softmax(tf.convert_to_tensor([1,2], dtype=tf.float32)))

Tensor("Reshape_550:0", shape=(2,), dtype=float32)


In [19]:
def ff(a):
    def _ff(b):
        return a + b
    return _ff

In [20]:
a = ff(4)

In [23]:
print(a)

<function ff.<locals>._ff at 0x7fee89f202f0>
